# Edge SleepFM-Lite: Colab Setup\n\nThiết lập môi trường và biến dùng chung cho pipeline.

In [ ]:
!pip install -q numpy pandas pyarrow h5py

In [ ]:
from google.colab import drivedrive.mount('/content/drive')

In [ ]:
import osPROJECT_DIR = '/content/drive/MyDrive/edge-sleepfm-lite'RAW_DIR = os.path.join(PROJECT_DIR, 'data', 'raw')HDF5_DIR = os.path.join(PROJECT_DIR, 'data', 'canonical_hdf5')PARQUET_DIR = os.path.join(PROJECT_DIR, 'data', 'training_store')SPLIT_PATH = os.path.join(PROJECT_DIR, 'data', 'splits', 'split.json')TEACHER_TARGET_DIR = os.path.join(PROJECT_DIR, 'data', 'teacher_targets')CHECKPOINT_DIR = os.path.join(PROJECT_DIR, 'checkpoints')print('PROJECT_DIR:', PROJECT_DIR)print('RAW_DIR:', RAW_DIR)print('HDF5_DIR:', HDF5_DIR)print('PARQUET_DIR:', PARQUET_DIR)print('SPLIT_PATH:', SPLIT_PATH)print('TEACHER_TARGET_DIR:', TEACHER_TARGET_DIR)print('CHECKPOINT_DIR:', CHECKPOINT_DIR)